<a href="https://colab.research.google.com/github/maheshboj/agenticai_basics/blob/Langchain_components/Snowflake_DB_Agent_langsmith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Install Required Packages
---



In [1]:
%pip install langchain  langgraph  langchain-community snowflake-connector-python snowflake-sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pyopenssl to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.8 MB/s eta 

In [4]:
%pip install -qU langsmith python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.8/411.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.7/343.7 kB 20.1 MB/s eta 0:00:00


In [8]:
from dotenv import load_dotenv
import os
load_dotenv('/content/.env')

True

In [10]:
from google.colab import userdata


In [11]:
# You can set them inline!
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY2')
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API')
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-db_agent"

In [13]:
%pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 5.9 MB/s eta 0:00:00


In [17]:
%pip install pyyaml


In [15]:
from langchain.chat_models import init_chat_model
from langsmith import traceable

In [16]:
llm = init_chat_model(model='gpt-4o-mini')

In [19]:
import yaml

# Load YAML file
with open("snowflake_config.yaml", "r") as f:
    config = yaml.safe_load(f)

sf = config["snowflake"]

# Build Snowflake connection URI
connection_uri = (
    f"snowflake://{sf['user']}:{sf['password']}@{sf['account']}/"
    f"{sf['database']}/{sf['schema']}?warehouse={sf['warehouse']}"
)

# If role is included
if "role" in sf:
    connection_uri += f"&role={sf['role']}"

#print("Snowflake URI:", connection_uri)


In [20]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(connection_uri)


In [21]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [22]:
data=db.run("select * from SNOWFLAKE_LEARNING_DB.PUBLIC.SF_SALES_TABLE;")

In [23]:
print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_table_names()}")


Dialect: snowflake
Available tables: ['dim_customer', 'dim_product', 'sf_sales_table']


/tmp/ipython-input-4080726409.py:2: LangChainDeprecationWarning: The method `SQLDatabase.get_table_names` was deprecated in langchain-community 0.0.1 and will be removed in 1.0. Use `get_usable_table_names` instead.
  print(f"Available tables: {db.get_table_names()}")


In [24]:
system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=5,
)

In [25]:
from langchain.agents import create_agent

agent = create_agent(
    llm,
    tools,
    system_prompt=system_prompt,
)

In [26]:
question = "give me the row count for SF_SALES_TABLE;?"
agent.invoke({"messages": [{"role": "user", "content": question}]})

{'messages': [HumanMessage(content='give me the row count for SF_SALES_TABLE;?', additional_kwargs={}, response_metadata={}, id='eea34d85-e712-4aa8-8603-11b3773b4d25'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 531, 'total_tokens': 543, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_a460d7e2b7', 'id': 'chatcmpl-Cjjp8ywkWaUf1FBSPiJVpPqB2voVy', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--af4c1f8d-1cab-4566-a0b3-8b28c4685e23-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_b7A4hsYoqApijeBZve4qlaBl', 'type': 'tool_call'}], usage_metadata={'input_tokens': 531, 'output_tokens': 12, '

In [28]:
@traceable
def run_agent(question):
    return agent.invoke({"messages": [{"role": "user", "content": question}]})

In [29]:
question = "give me the row count for SF_SALES_TABLE;?"
result=run_agent(question)

In [ ]:
question = "give me the row count for SF_SALES_TABLE;?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

give me the row count for SF_SALES_TABLE;?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_BvQoUAtVosFORnam2wL5AAyp)
 Call ID: call_BvQoUAtVosFORnam2wL5AAyp
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

dim_customer, dim_product, sf_sales_table
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_wBrK7ce1RQ7QLPDJQu4piQcX)
 Call ID: call_wBrK7ce1RQ7QLPDJQu4piQcX
  Args:
    query: SELECT COUNT(*) AS row_count FROM sf_sales_table;
================================= Tool Message =================================
Name: sql_db_query

[(58168,)]
================================== Ai Message ==================================

The row count for the `SF_SALES_TABLE` is 58,168.


In [ ]:
question = "Which many tables do we have in this db?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which many tables do we have in this db?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_Xyl9xRpMKKgw5RFkkQoaj1e6)
 Call ID: call_Xyl9xRpMKKgw5RFkkQoaj1e6
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

dim_customer, dim_product, sf_sales_table
================================== Ai Message ==================================

There are three tables in the database: `dim_customer`, `dim_product`, and `sf_sales_table`.


In [ ]:
question = "how many unique products do we have in sales table"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

how many unique products do we have in sales table
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_FtSBBq1CHaqzPRjQWldzYh95)
 Call ID: call_FtSBBq1CHaqzPRjQWldzYh95
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

dim_customer, dim_product, sf_sales_table
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_XSJGwFUMhrgpXOiCtvye56Pa)
 Call ID: call_XSJGwFUMhrgpXOiCtvye56Pa
  Args:
    table_names: sf_sales_table
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE sf_sales_table (
	productkey DECIMAL(38, 0), 
	orderdatekey DECIMAL(38, 0), 
	duedatekey DECIMAL(38, 0), 
	shipdatekey DECIMAL(38, 0), 
	customerkey DECIMAL(38, 0), 
	salesordernumber VAR

In [ ]:
question = "give me the name of the customer with customer key 16942	 "

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

give me the name of the customer with customer key 16942	 
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_zMol3gfnJK7LonHFToog26ko)
 Call ID: call_zMol3gfnJK7LonHFToog26ko
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

dim_customer, dim_product, sf_sales_table
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_lftVieg9h6wTsTVme7S2XZKk)
 Call ID: call_lftVieg9h6wTsTVme7S2XZKk
  Args:
    table_names: dim_customer
  sql_db_schema (call_mTKxA4FIPvrZMELTTs3562On)
 Call ID: call_mTKxA4FIPvrZMELTTs3562On
  Args:
    table_names: dim_product
  sql_db_schema (call_UD5N5aGRzv3AW45465BImU1p)
 Call ID: call_UD5N5aGRzv3AW45465BImU1p
  Args:
    table_names: sf_sales_table
================================= Tool Messa

In [ ]:
question = "give list of all the products bought by Kaitlyn Bennett "

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

give list of all the products bought by Kaitlyn Bennett 
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_Mjnwm6umDG4VUY00Dxk1KQMp)
 Call ID: call_Mjnwm6umDG4VUY00Dxk1KQMp
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

dim_customer, dim_product, sf_sales_table
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_sN1ytF91jInDPhPUwOhNKhjK)
 Call ID: call_sN1ytF91jInDPhPUwOhNKhjK
  Args:
    table_names: dim_customer
  sql_db_schema (call_lfp70hXHbzFUD7a3aTgcdz4b)
 Call ID: call_lfp70hXHbzFUD7a3aTgcdz4b
  Args:
    table_names: dim_product
  sql_db_schema (call_g9SOE05KHEVQLrer6bbf2CWF)
 Call ID: call_g9SOE05KHEVQLrer6bbf2CWF
  Args:
    table_names: sf_sales_table
================================= Tool Message